# Qualitative analysis of Compression

In [1]:
import pandas as pd
import json
import numpy as np
from shapely.geometry import Point, LineString, Polygon
from shapely.ops import nearest_points
import folium
import subprocess
from io import StringIO
from dtaidistance import dtw_ndim

from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from geopy.distance import geodesic 
from pyproj import Geod
from shapely.ops import nearest_points
import json

### Trajectory loading

In [2]:
FILE = "cars_8.csv"
df = pd.read_csv(f"datasets/{FILE}")
df = df.rename(columns={"Longitude": "longitude", "Latitude": "latitude", "Timestamp": "time"})
df['time'] = pd.to_datetime(df['time'])
df['time'] = df["time"].values.astype(np.int64)
df = df.sort_values(by=['id', 'time'])
df = df[["id","time","latitude","longitude"]]
df.to_csv(f"datasets/movetk_dataset/{FILE}",index=False)
df

,id,time,latitude,longitude
0,8,1591001768148000000,50.861073,4.465373
1,8,1591001769648000000,50.861100,4.465386
2,8,1591001770398000000,50.861127,4.465398
3,8,1591001770797745000,50.861146,4.465406
4,8,1591001784229685000,50.861756,4.465684
...,...,...,...,...
4937,8,1591307819514958000,50.861842,4.465724
4938,8,1591307819943340000,50.861822,4.465716
4939,8,1591307821415195000,50.861756,4.465684
4940,8,1591307834847135000,50.861146,4.465406


### Compression

In [3]:
TOLERANCE = 10
a = subprocess.run(["/home/pierre-cedric/Documents/MEMO-F524/movetk/build/examples/simplify-douglas", f"/home/pierre-cedric/Documents/MEMO-F524/datasets/movetk_dataset/{FILE}", str(TOLERANCE/111139)], capture_output=True)
txt = StringIO(a.stdout.decode("utf-8"))
compressed_df = pd.read_csv(txt, sep=",")
#cleaned_df = cleaned_df.rename(columns={"Longitude": "longitude", "Latitude": "latitude", "Timestamp": "time"})
#cleaned_df["time"] = cleaned_df["Timestamp"]
compressed_df

,id,longitude,latitude
0,8,4.465374,50.861073
1,8,4.465744,50.861885
2,8,4.464193,50.862209
3,8,4.462911,50.863354
4,8,4.459708,50.864429
...,...,...,...
116,8,4.459630,50.864395
117,8,4.462911,50.863354
118,8,4.464263,50.862175
119,8,4.465744,50.861885


In [14]:
series1 = df[["id","latitude","longitude"]].values
series2 = compressed_df[["id","latitude","longitude"]].values
series1

array([[ 8.        , 50.8610729 ,  4.4653735 ],
       [ 8.        , 50.86110017,  4.4653858 ],
       [ 8.        , 50.86112743,  4.4653981 ],
       ...,
       [ 8.        , 50.8617555 ,  4.4656845 ],
       [ 8.        , 50.8611456 ,  4.4654063 ],
       [ 8.        , 50.8610729 ,  4.4653735 ]])

In [17]:
def dist(coord1, coord2):
    return geodesic(coord1, coord2).m
def compute_ED(initial, compressed):
    ED = []
    A = initial[0]
    compressed_path = LineString([Point(p[1], p[2]) for p in compressed]) 
    for i in range(len(initial)):
        B =  initial[i]
        point_B = Point(B[1], B[2])
        _point_B = compressed_path.interpolate(compressed_path.project(point_B))
        
        ED.append(dist((_point_B.x,_point_B.y), ((B[1], B[2]))))
    return ED
data = compute_ED(series1, series2)
sum(data)/len(data)

2.1294920738668828

In [5]:
import numpy as np
step = 16
dates = [el for el in range(1703340887,1703340887+len(compressed_df)*step,step)]
times = np.array(dates, dtype='datetime64[s]')
times
compressed_df["time"] = times
compressed_df

,id,longitude,latitude,time
0,8,4.465374,50.861073,2023-12-23 14:14:47
1,8,4.465744,50.861885,2023-12-23 14:15:03
2,8,4.464193,50.862209,2023-12-23 14:15:19
3,8,4.462911,50.863354,2023-12-23 14:15:35
4,8,4.459708,50.864429,2023-12-23 14:15:51
...,...,...,...,...
116,8,4.459630,50.864395,2023-12-23 14:45:43
117,8,4.462911,50.863354,2023-12-23 14:45:59
118,8,4.464263,50.862175,2023-12-23 14:46:15
119,8,4.465744,50.861885,2023-12-23 14:46:31


In [6]:
compressed_df.to_csv("csv_results/movetk-comp-results.csv")